In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizVert_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,179861.32,-2.87,-2.40,-1.88,-0.30,-0.25,0.09,-0.11,-0.00
3,-2.87,48256.45,-0.78,0.31,-0.00,-0.06,-0.04,0.02,0.02
4,-2.40,-0.78,13508.84,0.06,0.21,-0.01,-0.04,-0.00,-0.00
5,-1.88,0.31,0.06,3325.70,-0.01,-0.01,-0.01,-0.01,-0.01
6,-0.30,-0.00,0.21,-0.01,905.79,0.01,0.00,0.00,0.00
7,-0.25,-0.06,-0.01,-0.01,0.01,251.18,0.00,0.00,-0.00
8,0.09,-0.04,-0.04,-0.01,0.00,0.00,57.71,0.00,-0.00
9,-0.11,0.02,-0.00,-0.01,0.00,0.00,0.00,14.39,-0.00
10,-0.00,0.02,-0.00,-0.01,0.00,-0.00,-0.00,-0.00,2.16


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00003,-0.00005,-0.00008,-0.00002,-0.00004,0.00003,-0.00007,-0.00000
3,-0.00003,1.00000,-0.00003,0.00002,-0.00000,-0.00002,-0.00002,0.00002,0.00005
4,-0.00005,-0.00003,1.00000,0.00001,0.00006,-0.00001,-0.00005,-0.00001,-0.00001
5,-0.00008,0.00002,0.00001,1.00000,-0.00000,-0.00002,-0.00002,-0.00005,-0.00007
6,-0.00002,-0.00000,0.00006,-0.00000,1.00000,0.00002,0.00001,0.00002,0.00003
7,-0.00004,-0.00002,-0.00001,-0.00002,0.00002,1.00000,0.00002,0.00007,-0.00001
8,0.00003,-0.00002,-0.00005,-0.00002,0.00001,0.00002,1.00000,0.00002,-0.00001
9,-0.00007,0.00002,-0.00001,-0.00005,0.00002,0.00007,0.00002,1.00000,-0.00004
10,-0.00000,0.00005,-0.00001,-0.00007,0.00003,-0.00001,-0.00001,-0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

6.072521648655125

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.59918217e+09 2.54867263e+08 1.95543437e+07 1.15391961e+06
 8.22248075e+04 5.92184650e+03 2.80423806e+02 1.30171244e+01
 5.59757128e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999303,-0.036056,-0.009345,-0.002294,-0.000624,-0.000174,-0.000039,-0.000010,-0.000001
1,0.035589,0.998361,-0.043645,-0.009824,-0.002665,-0.000740,-0.000170,-0.000042,-0.000006
2,0.010767,0.042779,0.997988,-0.044024,-0.011187,-0.003092,-0.000713,-0.000177,-0.000027
3,0.003061,0.011403,0.042754,0.997149,-0.059058,-0.015030,-0.003432,-0.000858,-0.000130
4,0.000996,0.003704,0.013261,0.057139,0.995281,-0.075471,-0.015758,-0.003909,-0.000585
5,0.000347,0.001290,0.004603,0.018780,0.072836,0.993638,-0.081456,-0.018946,-0.002832
6,0.000103,0.000387,0.001380,0.005599,0.020598,0.076810,0.987950,-0.131438,-0.017931
7,0.000040,0.000145,0.000519,0.002110,0.007731,0.027914,0.125278,0.979994,-0.151902
8,0.000012,0.000044,0.000158,0.000643,0.002351,0.008485,0.036939,0.148195,0.988229


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = np.sort(np.concatenate(cos_dist))[:len(GROUPS)]
cos_dist


array([0.00069676, 0.00163919, 0.0020117 , 0.00285086, 0.00471872,
       0.00636166, 0.01177139, 0.01205006, 0.02000649])